<a href="https://colab.research.google.com/github/yeega185/python-RPA-lab/blob/main/weather_api_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests              # 安裝API套件
import csv                # 把資料寫進CSV檔案
from datetime import datetime, timedelta      # 取得今天日期與時間

API_KEY = "2ee72ba9f03b56b7620f04b835f39854"        # <-- 換成你的 OpenWeatherMap API key
CITIES = ["Taipei,tw", "Taoyuan,tw", "Hsinchu,tw", "Taichung,tw", "Tainan,tw", "Kaohsiung,tw"]   # 多個城市查詢 台北、桃園、新竹、台中、台南、高雄


url = f"https://api.openweathermap.org/data/2.5/weather?q={CITIES}&appid={API_KEY}&units=metric&lang=zh_tw"

now = datetime.now().strftime('%Y-%m-%d %H:%M')               #現在的日期
tomorrow_str = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d %H:%M')  #明天的日期
rows = []

print(now)
print(tomorrow_str)
print(url)




2025-07-24 16:38
2025-07-25 16:38
https://api.openweathermap.org/data/2.5/weather?q=['Taipei,tw', 'Taoyuan,tw', 'Hsinchu,tw', 'Taichung,tw', 'Tainan,tw', 'Kaohsiung,tw']&appid=2ee72ba9f03b56b7620f04b835f39854&units=metric&lang=zh_tw


In [2]:
for city in CITIES:
  # 1. 抓現在天氣
  url_now = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=zh_tw"
  now_data = requests.get(url_now).json()
  now_weather = now_data['weather'][0]['description'] if 'weather' in now_data else 'N/A'
  now_temp = now_data['main']['temp'] if 'main' in now_data else 'N/A'

  # 2. 抓5天3小時預報
  url_fc = f"https://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric&lang=zh_tw"
  fc_data = requests.get(url_fc).json()
  # 三小時後
  three_hr = fc_data['list'][0]
  th_weather = three_hr['weather'][0]['description']
  th_temp = three_hr['main']['temp']
  # 明天12:00
  tomorrow_weather = next((item for item in fc_data['list'] if tomorrow_str in item['dt_txt'] and "12:00:00" in item['dt_txt']), None)
  if tomorrow_weather:
      tm_weather = tomorrow_weather['weather'][0]['description']
      tm_temp = tomorrow_weather['main']['temp']
  else:
      tm_weather = tm_temp = 'N/A'
  # 這週每天12:00
  week_fc = [item for item in fc_data['list'] if "12:00:00" in item['dt_txt']]
  # 儲存/印出
  print(f"{city}：")
  print(f"  現在天氣：{now_weather}，{now_temp}°C")
  print(f"  三小時後：{th_weather}，{th_temp}°C")
  print(f"  明天12:00：{tm_weather}，{tm_temp}°C")
  print("  這週預報（每天12:00）：")
  for w in week_fc:
    day = w['dt_txt'].split(" ")[0]
    desc = w['weather'][0]['description']
    temp = w['main']['temp']
    print(f"    {day}：{desc}，{temp}°C")
  print()

  # 若你要存成csv，可以自己定義欄位，這裡只示範寫一筆資料
  rows.append([city, now_weather, now_temp, th_weather, th_temp, tm_weather, tm_temp])

Taipei,tw：
  現在天氣：多雲，27°C
  三小時後：小雨，27°C
  明天12:00：N/A，N/A°C
  這週預報（每天12:00）：
    2025-07-25：小雨，26.54°C
    2025-07-26：小雨，25.59°C
    2025-07-27：陰，多雲，26.07°C
    2025-07-28：小雨，27.67°C
    2025-07-29：小雨，27.69°C

Taoyuan,tw：
  現在天氣：多雲，26.53°C
  三小時後：小雨，26.53°C
  明天12:00：N/A，N/A°C
  這週預報（每天12:00）：
    2025-07-25：小雨，26.29°C
    2025-07-26：小雨，25.55°C
    2025-07-27：小雨，25.71°C
    2025-07-28：中雨，26.68°C
    2025-07-29：中雨，26.13°C

Hsinchu,tw：
  現在天氣：晴，少雲，27.16°C
  三小時後：小雨，27.16°C
  明天12:00：N/A，N/A°C
  這週預報（每天12:00）：
    2025-07-25：中雨，26.83°C
    2025-07-26：陰，多雲，26.8°C
    2025-07-27：小雨，25.94°C
    2025-07-28：中雨，26.93°C
    2025-07-29：小雨，27.6°C

Taichung,tw：
  現在天氣：多雲，27.7°C
  三小時後：中雨，27.7°C
  明天12:00：N/A，N/A°C
  這週預報（每天12:00）：
    2025-07-25：小雨，26.57°C
    2025-07-26：陰，多雲，26.24°C
    2025-07-27：小雨，24.89°C
    2025-07-28：小雨，27.37°C
    2025-07-29：小雨，25.64°C

Tainan,tw：
  現在天氣：多雲，27.95°C
  三小時後：多雲，27.95°C
  明天12:00：N/A，N/A°C
  這週預報（每天12:00）：
    2025-07-25：小雨，29.71°C
    2025-07-26：小雨，28.64°C
  

In [3]:
# 寫入 csv
with open("weather_summary.csv", "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["城市", "現在天氣", "現在溫度", "三小時後天氣", "三小時後溫度", "明天12:00天氣", "明天12:00溫度"])
    for row in rows:
        writer.writerow(row)

print("全部城市天氣摘要已寫入 weather_summary.csv")

全部城市天氣摘要已寫入 weather_summary.csv


查詢現在目錄的檔案

In [4]:
!ls /content

sample_data  weather_summary.csv


## 將製作完成的weater.csv **檔案自動上傳到onedrive** **粗體文字**

安狀rclone

In [5]:
!curl https://rclone.org/install.sh | sudo bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4734  100  4734    0     0   6252      0 --:--:-- --:--:-- --:--:--  6253
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/git-log.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/rclone  [binary]
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/README.html  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.70.3-linux-amd64/rclone.1  [text]  
Purging old database entries in /usr/share/man...
Processing manual pages under /usr/share/man...
Updating index cache for path `/usr/share/man/man1'. Wait...done.
Checking for stray cats under /usr/share/man...
Checking for stray

利用rclone連線至onedrive

In [6]:
import os

rclone_config = """
[onedrive]
type = onedrive
token = {"access_token":"EwBIBMl6BAAUBKgm8k1UswUNwklmy2v7U/S+1fEAAb9LYkf03ml5Ehf5YtJ6BN+tIwdFXWng9M9/pEEkoOZIUSN+1oKoNe3d7FdbS0B6WzuZIbfxR7cO4z8dKDMB0U1BAvtaIUQZXCGDTFG1NblYaB2n3hPBFQHuZFa0VOIFCL7Wx7P7VE9LJXm0tdaCjsZPXoMnRTwGV/ha2orY40kqMPpjmXDTHQclm4Q3vgrXXyBhIilcyu3gRRN4YdIUfgAQ65/OlHB8iMzKEPpgLHVPKrR476edOXhNfl5fpujUXh7k2wYiZtvpPg/jI9bZEEqVI9I1Xd+Ea7XSSm5fo5lAP+c+ag7ANJbQcASFbzqF6S2h3PvUxj/8EG2sDB5oDn8QZgAAEOVSCSaGhbv8bDZo1nItTGcQAxzC9cpOxfNuY1qBXbnbqm3RM76LqCalJPgULMrIx9/tr47J8vxcuNJ+WndBbDp/atcOYX+rCtKCDFHVpFI7uDW/xzC0CJ49tx1r1rVWw2TsFIJHrEF/CZi9Z+7F3TvOSSvnark76kBu6zPH0/ukqBXKinMuhbHOEZNiVTcesNo3sxtpgyYvfac0Fcu6TaYuCV/7iMRTv8BjxsKPdPAgWXrnspVs+aEomXQLtYa5MmAhDNWxA8R8P0buGZJlbqsRQ+HTEUS0DYMc6yz79nt7FOp7UdBZAARkrB4XaIQ8+cyO/wW3usXRVYEXohngg8tHqpgiZ3sXrF5NjKLjkDn/ei4QYRmVxmnCifO/oAcAu/yC9Cd0O+vMMHeN4EQejNmGGlkvmqJS9a52VD58BYcBWlnNGUU3exLL9POmV3hSql3p95WLZrQO6aQkKUSSBuFaXUYU+MiVG9rg7AlcE+vJeSmjLZKC7ftFZR8RwEkgkqh3rTez0l75A30UzMDEnNqUaJpGPdAAZMR0JzyrQRhMKUuHTHBX3Smt2usKCiuATtKBTG+UPZKFf1IardkEIwc2nKpXrUMRJBvgipTQ8r7k4efIoWkNb+zaxywGjuauKh9C6eK3kTvKuLhEQvfFjyp050JaCPSAjMyVikjiPCZbzIuqJSH0fA1W/kaELx8RGD2sP4+Thx0LoYF3lbh1LZZqxB7wi9SKCEsFaDyhwcPur5TQPm7KdEnSROsDgPH5z9EpoAtnh4vq2xWaVoruyz+Asj5WoAnLeSAFzxXoIfePAWABBnqFaYmMQb5AnRyY/0Qrrv2bIsvMaO7yb2Sncz8qgRUFHCRhKKPk3nOBQ1gDWYnVV/OnoGvy+EZgWLZu7TJoC/W715xJXZjbSqTOgIF3TRLMLcY1HodDQylDJkoi9EYluAHpdUesZhVoHNcYtUIQMBLR4RaLuBD+Y5BxKDgSkWTBir4YnzRpIhuCCeMnHf/AKMJ72Ds9HNQswb8MxpSnmwFwydEu+wX1CnpS+2acUcVSVuU8o1XL/5h21WMsBQBCAw==","token_type":"Bearer","refresh_token":"M.C547_BAY.0.U.-ClamVdbC1AY*ahSG1hWXN6wWHO0Q1vN2JJ*R3iLObutw4580bD!OcJvXzIc0UuU*EQI3rm*zEQirdLI4ob0wlU9RD4aXVG78QSUBTimuKsY1CMkPLbrBz71RkgGt!Y7IgOIOHUN9AvV5ls8vDMDN7j*q4NUOGr7A56ZumTgWdvXiwGcvAdzq8G0m8z9VWgZrIInksO!5ssneyXxSYc4PAdzdEh1PreZoWvosZp8oKGZEOWcnR8UZo0cyOBmsMN0HrfleSGctV*OXRdap9d1goHk7mKLSuQEdAGnW4CTEccVSngyl7QoDyM!1Br0YEVhBBxYh6HoM70tfiK0VWvkn96bEWmLJS2dZS03aBuhUKagmkokvtQqjucAipuQuyly2yriwE!ZkiI!AuSYCTHEp4GZy3!xXqEg5!b9Z!BbmV4GFLsFGpHlOkhS7bFAFqBP1dg$$","expiry":"2025-07-24T22:03:36.9101941+08:00","expires_in":3599}
drive_id = 62DCE41F5DE89FFC
drive_type = personal
"""
os.makedirs('/root/.config/rclone/', exist_ok=True)
with open('/root/.config/rclone/rclone.conf', 'w') as f:
  f.write(rclone_config)






查詢Onedrive裡面的資料

In [ ]:
!rclone ls onedrive:/

In [8]:
!rclone copy weather_summary.csv onedrive:colab/

In [9]:
!rclone ls onedrive:colab/

      376 weather.csv
      398 weather_summary.csv
